# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [10]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv


importedggg.


#### Creating list of filepaths to process original event csv data files

In [11]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace
here


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [12]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [13]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [14]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
try:
    session = cluster.connect()
    print('connected')
except Exception as e: 
    print('Failed to connect',e)

connected


#### Create Keyspace

In [15]:
try:
    session.execute("""Create keyspace if not exists brian_udacity with replication = {'class': 'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print('failed',e)

#### Set Keyspace

In [16]:
try:
    session.set_keyspace('brian_udacity')
except Exception as e:
    print('failed',e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [17]:
#For table music_lib, session_id and session_item columns were used as partition key because the queries will filter by these column. 
drop_table = "Drop table if exists music_lib"
session.execute(drop_table)
qrr = """create table if not exists music_lib(
            artist text, 
            song text,
            song_length float,
            session_id int, 
            session_item int, 
        primary key(session_id,session_item))"""
try:
    session.execute(drop_table)
    session.execute(qrr)
except Exception as e:
    print('abort failed', e)

In [18]:
file = 'event_datafile_new.csv'
print('here')

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_lib(artist, song, song_length,session_id,session_item)"
        query = query + "values(%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3]) ))

here


#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
"""
Here we get all records based on our usecase
"""
get_data = "select artist,song,song_length,session_id,session_item from music_lib where session_id=338 AND session_item=4" 
recs = session.execute(get_data)
for row in recs:
    print(row.artist, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [22]:
#For table music_lib_user, user_id and session_id columns were used as partition key because the queries will filter by these column.
# This table uses session_item column as the clustering key which is used to order the results in DESC.
drop_table = "Drop table if exists music_lib_user"
qrr = """CREATE TABLE if not exists music_lib_user (
             artist text, 
             song text,
             f_name text,
             l_name text, 
             user_id int,
             session_id int, 
             session_item int,  
         PRIMARY KEY ((user_id,session_id),session_item) 
) WITH CLUSTERING ORDER BY (session_item DESC);"""
try:
    session.execute(drop_table)
    session.execute(qrr)
except Exception as e:
    print('abort failed', e)

                    

In [23]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_lib_user(artist, song, f_name,l_name,user_id,session_id,session_item)"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], line[1], line[4],int(line[10]), int(line[8]), int(line[3]) ))

In [24]:
get_data = "select artist,song,f_name,l_name,session_id,session_item from music_lib_user where user_id = 10 and session_id = 182 ORDER BY session_item DESC" 
recs = session.execute(get_data)
for row in recs:
    print(row.artist, row.song,row.f_name,row.l_name,row.session_id)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 182
Sebastien Tellier Kilometer Sylvie Cruz 182
Three Drives Greece 2000 Sylvie Cruz 182
Down To The Bone Keep On Keepin' On Sylvie Cruz 182


In [26]:
#For table music_lib_user_song, song and user_id columns were used as partition key because the queries will filter by these column.
drop_table = "Drop table if exists music_lib_user_song"
qrr = """create table if not exists music_lib_user_song(
            song text,
            user_id int,
            f_name text,
            l_name text,
        primary key(song,user_id))"""
try:
    session.execute(drop_table)
    session.execute(qrr)
except Exception as e:
    print('abort failed', e)
                    

In [27]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_lib_user_song(song,user_id,f_name,l_name)"
        query = query + "values(%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [28]:
get_data = "select f_name,l_name from music_lib_user_song where song = 'All Hands Against His Own'" 
recs = session.execute(get_data)
for row in recs:
    print(row.f_name,row.l_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [29]:
drop_table_lib="""Drop table music_lib"""
drop_table_libuser="""Drop table music_lib_user"""
drop_table_libusersong="""Drop table music_lib_user_song"""
try:
    session.execute(drop_table_lib)
    session.execute(drop_table_libuser)
    session.execute(drop_table_libusersong)
except Exception as e:
    print('failed dropping tables review code', e)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()